In [12]:
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import plotly.graph_objects as go

global rx0, ry0, rz0, tx0, ty0, tz0, bx0, by0, bz0, kappa0


with open('data_lower_envelope.json', 'r') as file:
    numData = json.load(file)


# index of a band with n half twists. first column is n, 2nd, 3rd, and 4th are
# index_min,index_optimal, and index_max

n_index = [[3, 1, 1, 24],
        [5, 25, 70, 93],
        [7, 94, 135, 160],
        [9, 161, 199, 224],
        [11, 225, 262, 288],
        [13, 289, 325, 351],
        [15, 352, 388, 414],
        [17, 415, 449, 476],
        [19, 477, 511, 539],
        [21, 540, 573, 600],
        [23, 601, 634, 663],
        [25, 663, 695, 723],
        [27, 723, 756, 770],
        [27, 770, 756, 770]]

##################################################
##################################################

ind_opt = [row[2] for row in n_index]  # index of optimal bands

# for eversion, we use time like parameter t

ind = 1 # index for which we want to find the variable


numArray = numData[ind-1]

N = int(math.floor((len(numArray) - 3) / 3))
N1 = N - 1
h = 1 / N

n = 3
nr = n

In [13]:
def data_t0(nu_i, N, numArray):
    
    global rx0, ry0, rz0, tx0, ty0, tz0, bx0, by0, bz0, kappa0

    h = 1/N

    tau = (1.29 + 0.015877763328999*(nu_i-1))*2*math.pi

    tx0 = np.transpose(np.zeros(N+1))
    ty0 = np.transpose(np.zeros(N+1))
    tz0 = np.transpose(np.zeros(N+1))

    bx0 = np.transpose(np.zeros(N+1))
    by0 = np.transpose(np.zeros(N+1))
    bz0 = np.transpose(np.zeros(N+1))

    rx0 = np.transpose(np.zeros(N+1))
    ry0 = np.transpose(np.zeros(N+1))
    rz0 = np.transpose(np.zeros(N+1))

    bx2p = np.transpose(np.zeros(N+1))
    by2p = np.transpose(np.zeros(N+1))
    bz2p = np.transpose(np.zeros(N+1))

    kappa0 = np.transpose(np.zeros(N+1))

    bx0[0] = numArray[0]
    by0[0] = numArray[1]
    bz0[0] = numArray[2]

    for i in range(1, N + 1):
        bx0[i] = numArray[i * 3]
        by0[i] = numArray[i * 3 + 1]
        bz0[i] = numArray[i * 3 + 2]

        tx0[i] = (by0[i - 1] * bz0[i] - by0[i] * bz0[i - 1])/tau/h
        ty0[i] = (bz0[i - 1] * bx0[i] - bz0[i] * bx0[i - 1])/tau/h
        tz0[i] = (bx0[i - 1] * by0[i] - bx0[i] * by0[i - 1])/tau/h

    tx0[0] = tx0[N]
    ty0[0] = ty0[N]
    tz0[0] = tz0[N]

    bx2p[0] = (bx0[1] - bx0[N - 1] - 2 * bx0[0]) / h ** 2
    by2p[0] = (by0[1] - by0[N - 1] - 2 * by0[0]) / h ** 2
    bz2p[0] = (bz0[1] - bz0[N - 1] - 2 * bz0[0]) / h ** 2

    bx2p[N] = -bx2p[0]
    by2p[N] = -by2p[0]
    bz2p[N] = -bz2p[0]

    kappa0[0] = tx0[0] * bx2p[0] + ty0[0] * by2p[0] + tz0[0] * bz2p[0]

    for i in range(1, N):
        bx2p[i] = (bx0[i + 1] + bx0[i - 1] - 2 * bx0[i]) / h ** 2
        by2p[i] = (by0[i + 1] + by0[i - 1] - 2 * by0[i]) / h ** 2
        bz2p[i] = (bz0[i + 1] + bz0[i - 1] - 2 * bz0[i]) / h ** 2

        kappa0[i] = tx0[i] * bx2p[i] + ty0[i] * by2p[i] + tz0[i] * bz2p[i]

    kappa0[N] = tx0[N] * bx2p[N] + ty0[N] * by2p[N] + tz0[N] * bz2p[N]

    kappa0 = [x / tau for x in kappa0]

    # Assuming rx0, ry0, rz0, tx0, ty0, tz0, cen are already defined
    rx0[0] = 0.0
    ry0[0] = 0.0
    rz0[0] = 0.0
    h = 1
    for i in range(N):
        rx0[i + 1] = rx0[i] + h * tx0[i + 1]
        ry0[i + 1] = ry0[i] + h * ty0[i + 1]
        rz0[i + 1] = rz0[i] + h * tz0[i + 1]

    # Assuming np.mean is a function defined elsewhere that computes average
    cen = [0, 0, 0]
    cen[0] = np.mean(rx0)
    cen[1] = np.mean(ry0)
    cen[2] = np.mean(rz0)

    for i in range(N + 1):
        rx0[i] = rx0[i] - cen[0]
        ry0[i] = ry0[i] - cen[1]
        rz0[i] = rz0[i] - cen[2]

    # Normalizing the midline
    dl = 0
    for i in range(N):
        dx = rx0[i + 1] - rx0[i]
        dy = ry0[i + 1] - ry0[i]
        dz = rz0[i + 1] - rz0[i]
        dl += math.sqrt(dx * dx + dy * dy + dz * dz)
    dl = dl / 6
    for i in range(N + 1):
        rx0[i] /= dl
        ry0[i] /= dl
        rz0[i] /= dl

    return None

def animationData(j):
    #ind = j % (2*N)  # ensuring that index is not out of bound
    ind2 = j % N  # ensuring that index is not out of bound
    ind = ind2
    integer_div = j // N
    if ind == 0:
        ind = 1
    if ind2 == 0:
        ind2 = 1

    or_value = -1

     
    ar1 = np.concatenate((bx0[ind2 - 1:N], bx0[0:ind2] * or_value))
    bx = ar1

    ar1 = np.concatenate((by0[ind2 - 1:N], by0[0:ind2] * or_value))
    by = ar1

    ar1 = np.concatenate((bz0[ind2 - 1:N], bz0[0:ind2] * or_value))
    bz = ar1

    ar1 = np.concatenate((kappa0[ind2 - 1:N], kappa0[0:ind2] * or_value))
    kappa = ar1

    ar1 = np.concatenate((rx0[ind2 - 1:N], rx0[0:ind2]))
    rx = ar1

    ar1 = np.concatenate((ry0[ind2 - 1:N], ry0[0:ind2]))
    ry = ar1

    ar1 = np.concatenate((rz0[ind2 - 1:N], rz0[0:ind2]))
    rz = ar1

     

    # ensuring that centroid is at the origin
    cen = [np.mean(rx), np.mean(ry), np.mean(rz)]
    rx -= cen[0]
    ry -= cen[1]
    rz -= cen[2]

    # rotating appropriately
    th = -math.acos(rx[0] / math.sqrt(rx[0] * rx[0] + ry[0] * ry[0]))

    if ry[0] < 0:
        th = 2 * math.pi - th

    if ry[0] == 0:
        if rx[0] > 0:
            th = 0
        else:
            th = math.pi

    ux = 0
    uy = 0
    uz = 1

    R1u = np.array([
        [math.cos(th) + ux ** 2 * (1 - math.cos(th)), ux * uy * (1 - math.cos(th)) - uz * math.sin(th),
         ux * uz * (1 - math.cos(th)) + uy * math.sin(th)],
        [ux * uy * (1 - math.cos(th)) + uz * math.sin(th), math.cos(th) + uy ** 2 * (1 - math.cos(th)),
         uy * uz * (1 - math.cos(th)) - ux * math.sin(th)],
        [ux * uz * (1 - math.cos(th)) - uy * math.sin(th), uy * uz * (1 - math.cos(th)) + ux * math.sin(th),
         math.cos(th) + uz ** 2 * (1 - math.cos(th))]
    ])

    for i in range(N + 1):
        temp = np.dot(R1u, [bx[i], by[i], bz[i]])
        bx[i], by[i], bz[i] = temp

        temp = np.dot(R1u, [rx[i], ry[i], rz[i]])
        rx[i], ry[i], rz[i] = temp

    return rx, ry, rz, bx, by, bz, kappa

In [14]:
def path_eversion():
    global n, N

    ind_frame = np.linspace(0, N, N, endpoint=True).astype(int)

    temp_rx = np.zeros(len(ind_frame) - 2)  # Adjusted size
    temp_ry = np.zeros(len(ind_frame) - 2)
    temp_rz = np.zeros(len(ind_frame) - 2)
    temp_bx = np.zeros(len(ind_frame) - 2)  # Adjusted size
    temp_by = np.zeros(len(ind_frame) - 2)
    temp_bz = np.zeros(len(ind_frame) - 2)
    
    for i in range(1, len(ind_frame) - 1):  # Adjusted range
        rx, ry, rz, bx, by, bz, kappa = animationData(ind_frame[i])
        index = int(np.round(N/n))
        temp_rx[i-1] = rx[index]
        temp_ry[i-1] = ry[index]
        temp_rz[i-1] = rz[index]

        temp_bx[i-1] = bx[index]
        temp_by[i-1] = by[index]
        temp_bz[i-1] = bz[index]
     
    temp_rx,temp_ry,temp_rz,temp_bx,temp_by,temp_bz = fun_pca(temp_rx,temp_ry,temp_rz,temp_bx,temp_by,temp_bz)
    return temp_rx,temp_ry,temp_rz,temp_bx,temp_by,temp_bz  # Return values

 
def fit_plane(points):
    n = points.shape[0]  # number of points
    # compute centroid
    center = np.mean(points, axis=0)
    

    # compute the covariance matrix
    cov_pts = np.cov(points, rowvar=False) / n

    # perform a principal component analysis with 2 variables,
    # to extract inertia axes
    U, S, _ = np.linalg.svd(cov_pts)

    # sort axes from greater to lower
    ind = np.argsort(-S)

    # format U to ensure first axis points to positive x direction
    U = U[:, ind]
    if U[0, 0] < 0:
        U = -U
        # keep matrix determinant positive
        U[:, 2] = -U[:, 2]

    plane = np.hstack((center, U[:, 0], U[:, 1]))
    normal = np.cross(U[:, 0], U[:, 1])
    #print(f"this is normal : {normal}")
    return normal


def fun_pca(rx, ry, rz, bx, by, bz):
    normal = fit_plane(np.column_stack((rx, ry, rz)))
    

    norm = np.sqrt(np.dot(normal, normal))
    a, b, c = normal / norm

    # Rotating the mid-plane to align with the x-y plane
    # Axis of rotation u = (ux, uy, uz)
    # u = n x i = (0, c, -b)
    ux, uy, uz = -b/np.sqrt(a**2 + b**2), a/np.sqrt(a**2 + b**2), 0

    th = -np.arccos(c)

    R1u = np.array([
        [np.cos(th) + ux**2 * (1 - np.cos(th)), ux*uy*(1-np.cos(th)) - uz*np.sin(th), ux*uz*(1-np.cos(th)) + uy*np.sin(th)],
        [ux*uy*(1-np.cos(th)) + uz*np.sin(th), np.cos(th) + uy**2 * (1 - np.cos(th)), uy*uz*(1-np.cos(th)) - ux*np.sin(th)],
        [ux*uz*(1-np.cos(th)) - uy*np.sin(th), uy*uz*(1-np.cos(th)) + ux*np.sin(th), np.cos(th) + uz**2 * (1 - np.cos(th))]
    ])

    N = len(rx) - 1
    for i in range(N + 1):
        rx[i], ry[i], rz[i] = np.dot(R1u, [rx[i], ry[i], rz[i]])
        bx[i], by[i], bz[i] = np.dot(R1u, [bx[i], by[i], bz[i]])

     

    return rx,ry,rz,bx,by,bz


# Usage:
# Assuming points is a NumPy array of shape (n, 3)
# plane = fit_plane(points)

# Usage:
#path_rx, path_ry, path_rz, path_bx, path_by, path_bz = path_eversion()



In [18]:
import plotly.subplots as sp


import numpy as np

def filter_data(x, y, z, threshold=0.15):
    """
    Filters the input arrays x, y, z to remove points for which the z-coordinate
    has a standard deviation greater than the specified threshold.

    Parameters:
    - x, y, z: Input arrays.
    - threshold: The percentage threshold for standard deviation filtering.

    Returns:
    - x_corrected, y_corrected, z_corrected: Filtered arrays.
    """
    # Compute the mean and standard deviation of the z-coordinate
    mean_z = np.mean(z)
    std_z = np.std(z)

    # Determine the points to keep based on the standard deviation threshold
    lower_bound = mean_z - threshold * std_z
    upper_bound = mean_z + threshold * std_z
    keep_points = (z >= lower_bound) & (z <= upper_bound)

    # Filter the x, y, and z arrays
    # x_corrected = x[keep_points]
    # y_corrected = y[keep_points]
    # z_corrected = z[keep_points]
    x_corrected = x
    y_corrected = y
    z_corrected = z

    return x_corrected, y_corrected, z_corrected

# Usage:
# Assuming x, y, z are your data arrays
# x_corrected, y_corrected, z_corrected = filter_data(x, y, z)



def plot3D(path_rx,path_ry,path_rz,label):
       

   trace = go.Scatter3d(x=path_rx, y=path_ry, z=path_rz,
                         mode='lines',
                         line=dict(width=4),
                         name = label)
   return trace
def plot2D(path_rx,path_ry, label):
   

   trace = go.Scatter3d(x=path_rx, y=path_ry,
                         mode='lines',
                         line=dict(width=4),
                         name = label)
   return trace

In [25]:
# now we plot the data for all of the optimal bands
# Define the number of rows and columns based on your requirements
rows = 1
cols = 1
nmax = 2

subplot_titles = [(f'n = {2*i + 1} twists') for i in range(1, nmax)]

fig = sp.make_subplots(
    rows=rows, cols=cols,
    subplot_titles=subplot_titles,
    specs=[[{'type': 'scatter3d'} for col in range(cols)] for row in range(rows)],
    vertical_spacing=0.01,  # Adjust vertical spacing between subplots
    horizontal_spacing=0.01  # Adjust horizontal spacing between subplots
)
global n
for i in range(1,nmax):
    # for eversion, we use time like parameter t
    print(f"index for the optimal band is {ind_opt[i-1]}")
    ind = 5
    numArray = numData[ind_opt[ind-1]-1]
    n = 2*ind+1
    N = int(math.floor((len(numArray) - 3) / 3))
    N1 = N - 1
    h = 1 / N
    print(f"number of points is {N}")

    data_t0(1, N, numArray)  # initializing data at time t0
    path_rx, path_ry, path_rz, path_bx, path_by, path_bz = path_eversion()
    
    # filter out erroneous points
    path_rx, path_ry, path_rz = filter_data(path_rx, path_ry, path_rz, threshold=0.9)
     
    legend_label = f'n = {2*i + 1}'  # Create the legend label
    trace = plot3D(path_rx, path_ry, path_rz, legend_label)  # Pass the legend label
    fig.add_trace(trace, row=((i-1)//cols) + 1, col=((i-1)%cols) + 1)

    # Update layout for scientific notation and display the plot
scene_config = {}

# Loop through each subplot and create a scene configuration for it
for i in range(1, rows * cols + 1):
    scene_key = f'scene{i}'
    scene_config[scene_key] = dict(
        xaxis_title='X',
        xaxis_exponentformat='e',
        yaxis_title='Y',
        yaxis_exponentformat='e',
        zaxis_title='Z',
        zaxis_exponentformat='e'
    )

# Update layout with the scene configurations
fig.update_layout(**scene_config)

# Reduce margin around the plot
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

# Display the plot
fig.show()
 






index for the optimal band is 1
number of points is 330


In [26]:
!pip install kaleido


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 MB 16.9 MB/s eta 0:00:0000:0100:01


In [27]:
!pwd

/Users/vikashchaurasia/Library/CloudStorage/OneDrive-Personal/Vikash_Documents/Isometric_deformation/Matlab_files/fixed_rotation_final/code


In [28]:
save_path = "/Users/vikashchaurasia/Library/CloudStorage/OneDrive-Personal/Vikash_Documents/Isometric_deformation/Matlab_files/fixed_rotation_final/data_inversion/"

file_name = save_path + "output.svg"

In [31]:
import plotly.io as pio

# ... (rest of your code, including creating the figure)

# Specify the file name and format
file_name = "output.svg"

# Export the figure
pio.write_image(fig, file_name)

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
